In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

In [2]:
def pil_to_cv(pil_image):
    open_cv_image = np.array(pil_image)
    return cv.cvtColor(open_cv_image, cv.COLOR_RGB2BGR)

In [3]:
def match_template_new(img, template, threshold=0.5):
    image = pil_to_cv(img)
    template_cv = pil_to_cv(template)

    image_gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    template_gray = cv.cvtColor(template_cv, cv.COLOR_BGR2GRAY)

    h, w = template_gray.shape
    image_plot = image.copy()
    rectangles = []

    # Template matching with limited scale
    scales = [0.6,0.8, 0.9, 1.0, 1.1]
    for scale in scales:
        image_gray = cv.resize(image_gray, (int(w * scale), int(h * scale)))
        resized_template = cv.resize(template_gray, (int(w * scale), int(h * scale)))
        h_resized, w_resized = resized_template.shape

        res = cv.matchTemplate(image_gray, resized_template, cv.TM_CCOEFF_NORMED)
        res_copy = res.copy()

        # Search threshold
        while True:
            min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res_copy)
            if max_val < threshold:
                break

            rectangles.append([max_loc[0], max_loc[1], w_resized, h_resized])
            res_copy[max_loc[1]:max_loc[1] + h_resized, max_loc[0]:max_loc[0] + w_resized] = 0

    rectangles, _ = cv.groupRectangles(rectangles, groupThreshold=1, eps=0.3)

    # Print rectangle
    cropped_image = None
    for (x, y, w, h) in rectangles:
        cv.rectangle(image_plot, (x, y), (x + w, y + h), (0, 255, 255), 2)
        # Crop the first detected rectangle
        if cropped_image is None:
            cropped_image = image[y:y+h, x:x+w]
        else:
            cropped_image = image[y:y+h, x:x+w]

    # Convert RGB
    image_rgb = cv.cvtColor(image_plot, cv.COLOR_BGR2RGB)

    # Display images
    fig, ax = plt.subplots(1, 4, figsize=(16, 4))

    # Input image
    ax[0].imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))
    ax[0].set_title('Input Image')
    ax[0].axis('off')

    # Template image
    ax[1].imshow(template_gray, cmap='gray')
    ax[1].set_title('Template Image')
    ax[1].axis('off')

    # Result image
    ax[2].imshow(image_rgb)
    ax[2].set_title('Result')
    ax[2].axis('off')

    # Cropped image
    if cropped_image is not None:
        cropped_rgb = cv.cvtColor(cropped_image, cv.COLOR_BGR2RGB)
        ax[3].imshow(cropped_rgb)
        ax[3].set_title('Cropped Match')
        ax[3].axis('off')
    else:
        ax[3].imshow(np.zeros((50, 50, 3), dtype=np.uint8))  # Display a blank image if no match found
        ax[3].set_title('No Match Found')
        ax[3].axis('off')

    plt.tight_layout()
    plt.show()

In [7]:
import os
def match(image, threshold=0.5):
    image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
    image_gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    
    templates = os.listdir('../asset/comparator')
    for template in templates:
        template_img = cv.imread('../asset/comparator/' + str(template))
        template_img = cv.cvtColor(template_img,cv.COLOR_RGB2BGR)
        template_gray = cv.cvtColor(template_img, cv.COLOR_BGR2GRAY)

        h, w = template_gray.shape
        for i in range(0,3):
            image_gray = cv.resize(image_gray, (w, h))
            resized_template = cv.resize(template_gray, (w, h))

        
            res = cv.matchTemplate(image_gray, resized_template, cv.TM_CCOEFF_NORMED)
            res_copy = res.copy()

            min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res_copy)
            # Search threshold
            
            if max_val >= threshold:
                return True
            
            resized_template = cv.rotate(resized_template,cv.ROTATE_90_CLOCKWISE)

    return False


In [8]:
from PIL import Image

img = cv.imread('../dataset\Cutting Weevil/20211011_160853 (Custom) (Custom).jpg')

bil = match(img)
print(bil)

True


In [ ]:
img = cv.imread('..\IMG_20211106_161141 (Custom).jpg')

bil = match(img)
print(bil)